Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = '/home/mattw/data/other/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Regression

In [16]:
batch_size = 128
l2_param = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_param*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

# fit model
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 317.514526
Minibatch accuracy: 4.7%
Validation accuracy: 13.2%
Minibatch loss at step 500: 1.081775
Minibatch accuracy: 79.7%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 1.109803
Minibatch accuracy: 78.1%
Validation accuracy: 78.8%
Minibatch loss at step 1500: 0.987913
Minibatch accuracy: 80.5%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 1.146214
Minibatch accuracy: 79.7%
Validation accuracy: 76.3%
Minibatch loss at step 2500: 1.059630
Minibatch accuracy: 74.2%
Validation accuracy: 77.7%
Minibatch loss at step 3000: 1.149229
Minibatch accuracy: 76.6%
Validation accuracy: 76.0%
Test accuracy: 83.3%


**Validation/Test Accuracy**

 - l2_param = 0: (78.8 / 86.2)
 - l2_param = 0.00001: (78.9 / 86.1)
 - l2_param = 0.0001: (79.2 / 86.7)
 - l2_param = 0.001: (81.8 / 88.5) 
 - l2_param = 0.01: (80.9 / 88.1)
 - l2_param = 0.1: (76.0 / 83.3)

## 2-Layer Network

In [22]:
batch_size = 128
num_hid_units = 1024
l2_param = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_units]))
  biases1 = tf.Variable(tf.zeros([num_hid_units]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_hid_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1 = tf.matmul(tf_train_dataset, weights1) + biases1
  logits = tf.matmul(tf.nn.relu(h1), weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_param*tf.nn.l2_loss(
    weights1) + l2_param*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

# fit model
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31837.630859
Minibatch accuracy: 12.5%
Validation accuracy: 36.7%
Minibatch loss at step 500: 1.371632
Minibatch accuracy: 79.7%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 1.416590
Minibatch accuracy: 80.5%
Validation accuracy: 79.6%
Minibatch loss at step 1500: 1.330797
Minibatch accuracy: 80.5%
Validation accuracy: 75.1%
Minibatch loss at step 2000: 1.411143
Minibatch accuracy: 81.2%
Validation accuracy: 74.4%
Minibatch loss at step 2500: 1.368796
Minibatch accuracy: 75.0%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 1.396768
Minibatch accuracy: 78.9%
Validation accuracy: 76.5%
Test accuracy: 83.3%


**Test Accuracy**

 - l2_param = 0: (81.8 / 88.4)
 - l2_param = 0.00001: (81.4 / 87.8)
 - l2_param = 0.0001: (81.9 / 89.1)
 - l2_param = 0.001: (86.8 / 92.8)
 - l2_param = 0.01: (83.6 / 90.0)
 - l2_param = 0.1: (76.5 / 83.3)

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
batch_size = 128
num_hid_units = 1024
l2_param = 0.0

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_units]))
  biases1 = tf.Variable(tf.zeros([num_hid_units]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_hid_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1 = tf.matmul(tf_train_dataset, weights1) + biases1
  logits = tf.matmul(tf.nn.relu(h1), weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_param*tf.nn.l2_loss(
    weights1) + l2_param*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

# truncate data to see overfitting
train_dataset_ = train_dataset[:1280,:]
train_labels_ = train_labels[:1280,:]

# fit model
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_[offset:(offset + batch_size), :]
    batch_labels = train_labels_[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 354.396942
Minibatch accuracy: 14.1%
Validation accuracy: 25.4%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Test accuracy: 84.7%


Each minibatch accuracy becomes 100%, but both the validation accuracy and the final test accuracy are lower than when using more data.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [29]:
batch_size = 128
num_hid_units = 1024
l2_param = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_units]))
  biases1 = tf.Variable(tf.zeros([num_hid_units]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_hid_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # LAYER 1
  h1 = tf.matmul(tf_train_dataset, weights1) + biases1
  # create a placeholder to use for dropout in training
  keep_prob1 = tf.placeholder(tf.float32)
  h1_drop = tf.nn.dropout(tf.nn.relu(h1), keep_prob1)
    
  # LAYER 2
  h2 = tf.matmul(h1_drop, weights2) + biases2
  # create a placeholder to use for dropout in training
  keep_prob2 = tf.placeholder(tf.float32)
  logits = tf.nn.dropout(h2, keep_prob2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_param*tf.nn.l2_loss(
    weights1) + l2_param*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

num_steps = 3001

# truncate data to see overfitting
# train_dataset_ = train_dataset[:1280,:]
# train_labels_ = train_labels[:1280,:]

# fit model
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob1 : 0.5, keep_prob2 : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 930.026245
Minibatch accuracy: 11.7%
Validation accuracy: 31.9%
Minibatch loss at step 500: 291.174438
Minibatch accuracy: 42.2%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 131.564865
Minibatch accuracy: 41.4%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 75.839546
Minibatch accuracy: 42.2%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 44.522045
Minibatch accuracy: 40.6%
Validation accuracy: 79.9%
Minibatch loss at step 2500: 27.675673
Minibatch accuracy: 50.8%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 17.069714
Minibatch accuracy: 45.3%
Validation accuracy: 83.1%
Test accuracy: 89.8%


In the overfitting case, val/test goes from 77.4 / 84.7 to 79.7 / 86.9

In the full data case with lambda_2 = 0.001, val/test goes from 86.8 / 92.8 to 83.1 / 89.8 (keep_prob = 0.5)

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## 2-Layer Model

In [7]:
batch_size = 128
num_hid_units = 1024

# set up computation graph
graph = tf.Graph()
with graph.as_default():

  # create placeholders for all parameters that may change while optimizing newtork architecture
  l2_param = tf.placeholder(tf.float32)
  keep_prob1 = tf.placeholder(tf.float32)
  keep_prob2 = tf.placeholder(tf.float32)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    
  # Define model parameters as Variables
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_units]))
  biases1 = tf.Variable(tf.zeros([num_hid_units]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_hid_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # LAYER 1
  x1 = tf.matmul(dataset, weights1) + biases1
  z1 = tf.nn.relu(x1)
  z1_drop = tf.nn.dropout(z1, keep_prob1)
      
  # LAYER 2
  x2 = tf.matmul(z1_drop, weights2) + biases2  
  logits = tf.nn.dropout(x2, keep_prob2)

  # LOSS LAYER
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, labels)) + l2_param*tf.nn.l2_loss(
    weights1) + l2_param*tf.nn.l2_loss(weights2)
  
  # Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the data
  prediction = tf.nn.softmax(logits)

# optimization parameters
num_steps = 3001

# fit model
with tf.Session(graph=graph) as session:
  
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {dataset : batch_data, labels : batch_labels, 
                 keep_prob1 : 1.0, keep_prob2 : 1.0, l2_param : 0.001}
    
    # run the graph
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    
    # print updates
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      feed_dict = {dataset : valid_dataset, labels : valid_labels, 
                   keep_prob1 : 1.0, keep_prob2 : 1.0, l2_param : 0.001}
      print("Validation accuracy: %.1f%%" % accuracy(
        prediction.eval(feed_dict=feed_dict), valid_labels))
    
  # print final test results
  feed_dict = {dataset : test_dataset, labels : test_labels, 
               keep_prob1 : 1.0, keep_prob2 : 1.0, l2_param : 0.001}
  print("Test accuracy: %.1f%%" % accuracy(
        prediction.eval(feed_dict=feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 702.312805
Minibatch accuracy: 6.2%
Validation accuracy: 32.2%
Minibatch loss at step 500: 205.738464
Minibatch accuracy: 82.0%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 113.480957
Minibatch accuracy: 82.8%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 68.307243
Minibatch accuracy: 88.3%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 41.371475
Minibatch accuracy: 85.2%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 25.253286
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 15.446515
Minibatch accuracy: 85.2%
Validation accuracy: 86.9%
Test accuracy: 93.3%


## 3-Layer Model

In [78]:
batch_size = 128
num_hid_units = 1024

# set up computation graph
graph = tf.Graph()
with graph.as_default():

  # create placeholders for all parameters that may change while optimizing newtork architecture
  l2_param = tf.placeholder(tf.float32)
  keep_prob = tf.placeholder(tf.float32)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    
  # Define model parameters as Variables
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_units], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([num_hid_units]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_hid_units, num_hid_units], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([num_hid_units]))
    
  weights3 = tf.Variable(
    tf.truncated_normal([num_hid_units, num_labels], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # LAYER 1
  x1 = tf.matmul(dataset, weights1) + biases1
  z1 = tf.nn.relu(x1)
#   z1 = tf.nn.dropout(z1, keep_prob)
      
  # LAYER 2
  x2 = tf.matmul(z1, weights2) + biases2  
  z2 = tf.nn.relu(x2)
#   z2 = tf.nn.dropout(z2, keep_prob)
    
  # LAYER 3
  x3 = tf.matmul(z2, weights3) + biases3  
#   logits = tf.nn.dropout(x3, keep_prob)
  logits = x3

  # LOSS LAYER
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, labels)) + l2_param*tf.nn.l2_loss(
    weights1) + l2_param*tf.nn.l2_loss(weights2) + l2_param*tf.nn.l2_loss(weights3)
  
  # Optimizer
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 2000, 0.96)
#   learning_rate = tf.Variable(0.1, trainable=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the data
  prediction = tf.nn.softmax(logits)

# optimization parameters
num_steps = 24001

# fit model
with tf.Session(graph=graph) as session:
  
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {dataset : batch_data, labels : batch_labels, 
                 keep_prob : 1.0, l2_param : 0.001}
    
    # run the graph
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    
    # print periodic updates
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      feed_dict = {dataset : valid_dataset, labels : valid_labels, 
                   keep_prob : 1.0, l2_param : 0.001}
      print("Validation accuracy: %.1f%%" % accuracy(
        prediction.eval(feed_dict=feed_dict), valid_labels))
    
  # print final test results
  feed_dict = {dataset : test_dataset, labels : test_labels, 
               keep_prob : 1.0, l2_param : 0.001}
  print("Test accuracy: %.1f%%" % accuracy(
        prediction.eval(feed_dict=feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 13.197136
Minibatch accuracy: 11.7%
Validation accuracy: 32.9%
Minibatch loss at step 1000: 6.328592
Minibatch accuracy: 85.9%
Validation accuracy: 87.3%
Minibatch loss at step 2000: 5.244860
Minibatch accuracy: 89.8%
Validation accuracy: 88.5%
Minibatch loss at step 3000: 4.371994
Minibatch accuracy: 89.8%
Validation accuracy: 88.9%
Minibatch loss at step 4000: 3.703155
Minibatch accuracy: 91.4%
Validation accuracy: 89.7%
Minibatch loss at step 5000: 3.114111
Minibatch accuracy: 92.2%
Validation accuracy: 89.8%
Minibatch loss at step 6000: 2.729416
Minibatch accuracy: 93.8%
Validation accuracy: 89.5%
Minibatch loss at step 7000: 2.278212
Minibatch accuracy: 93.8%
Validation accuracy: 89.9%
Minibatch loss at step 8000: 1.995524
Minibatch accuracy: 93.8%
Validation accuracy: 89.9%
Minibatch loss at step 9000: 1.804051
Minibatch accuracy: 89.8%
Validation accuracy: 90.2%
Minibatch loss at step 10000: 1.575782
Minibatch accuracy: 93.0%
Validation accu

## 4-Layer Model

In [86]:
batch_size = 128
num_hid_units1 = 2048
num_hid_units2 = 1024
num_hid_units3 = 512

# set up computation graph
graph = tf.Graph()
with graph.as_default():

  # create placeholders for all parameters that may change while optimizing newtork architecture
  l2_param = tf.placeholder(tf.float32)
  keep_prob = tf.placeholder(tf.float32)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    
  # Define model parameters as Variables
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid_units1], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([num_hid_units1]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([num_hid_units1, num_hid_units2], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([num_hid_units2]))
  
  weights3 = tf.Variable(
    tf.truncated_normal([num_hid_units2, num_hid_units3], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([num_hid_units3]))

  weights4 = tf.Variable(
    tf.truncated_normal([num_hid_units3, num_labels], stddev=0.1))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # LAYER 1
  x1 = tf.matmul(dataset, weights1) + biases1
  z1 = tf.nn.relu(x1)
  z1_drop = tf.nn.dropout(z1, keep_prob)
      
  # LAYER 2
  x2 = tf.matmul(z1_drop, weights2) + biases2  
  z2 = tf.nn.relu(x2)
  z2_drop = tf.nn.dropout(z2, keep_prob)

  # LAYER 3
  x3 = tf.matmul(z2_drop, weights3) + biases3  
  z3 = tf.nn.relu(x3)
  z3_drop = tf.nn.dropout(z3, keep_prob)
    
  # LAYER 4
  x4 = tf.matmul(z3_drop, weights4) + biases4  
  logits = tf.nn.dropout(x4, keep_prob)

  # LOSS LAYER
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, labels)) + l2_param*tf.nn.l2_loss(
    weights1) + l2_param*tf.nn.l2_loss(weights2) + l2_param*tf.nn.l2_loss(
    weights3) + l2_param*tf.nn.l2_loss(weights4)
  
  # Optimizer - learning rate decay
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 2000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Optimizer - no learning rate decay
#   learning_rate = tf.Variable(0.1, trainable=False)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the data
  prediction = tf.nn.softmax(logits)

# optimization parameters
num_steps = 24001

# fit model
with tf.Session(graph=graph) as session:
  
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {dataset : batch_data, labels : batch_labels, 
                 keep_prob : 1.0, l2_param : 0.001}
    
    # run the graph
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    
    # print periodic updates
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      feed_dict = {dataset : valid_dataset, labels : valid_labels, 
                   keep_prob : 1.0, l2_param : 0.0}
      print("Validation accuracy: %.1f%%" % accuracy(
        prediction.eval(feed_dict=feed_dict), valid_labels))
    
  # print final test results
  feed_dict = {dataset : test_dataset, labels : test_labels, 
               keep_prob : 1.0, l2_param : 0.0}
  print("Test accuracy: %.1f%%" % accuracy(
        prediction.eval(feed_dict=feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 28.379482
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 14.081381
Minibatch accuracy: 88.3%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 11.656403
Minibatch accuracy: 89.8%
Validation accuracy: 88.0%
Minibatch loss at step 3000: 9.622300
Minibatch accuracy: 90.6%
Validation accuracy: 88.6%
Minibatch loss at step 4000: 8.079641
Minibatch accuracy: 93.0%
Validation accuracy: 89.0%
Minibatch loss at step 5000: 6.730828
Minibatch accuracy: 94.5%
Validation accuracy: 89.4%
Minibatch loss at step 6000: 5.756310
Minibatch accuracy: 91.4%
Validation accuracy: 89.4%
Minibatch loss at step 7000: 4.821667
Minibatch accuracy: 94.5%
Validation accuracy: 89.7%
Minibatch loss at step 8000: 4.146935
Minibatch accuracy: 94.5%
Validation accuracy: 89.7%
Minibatch loss at step 9000: 3.594169
Minibatch accuracy: 90.6%
Validation accuracy: 90.2%
Minibatch loss at step 10000: 3.113681
Minibatch accuracy: 93.0%
Validation acc

# 1-Layer Models (Logistic Regression)
 - l2=0.001, 3000 batches: (81.8 / 88.5) 

# 2-Layer Models
 - 2-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 3000 batches: 86.8 / 92.8
 
# 3-Layer Models 
Increase training time to see how much this affects final performance
 - 3-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 3000 batches: 88.9 / 94.2
 - 3-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 6000 batches: 89.7 / 95.0
 - 3-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 9000 batches: 90.1 / 95.3

Using 6000 batches, make small tweaks to the network architecture to see if changes 
result in an obvious performance increase
 - 3-layer (1024 hidden units), l2=0.01,  keep_prob=1.0, 6000 batches: 85.3 / 91.3
 - 3-layer (2048 hidden units), l2=0.001, keep_prob=1.0, 6000 batches: 89.9 / 95.3 
 - 3-layer (2048 hidden units), l2=0.001, keep_prob=0.5, 6000 batches: 90.0 / 95.2

Run the best for a long time
 - 3-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 24000 batches: 90.1 / 95.5
 - 3-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 24000 batches, weight decay: **90.5 / 95.9**

# 4-Layer models
 - 4-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 9000 batches: 89.9 / 95.2
 - 4-layer (1024 hidden units), l2=0,     keep_prob=0.5, 9000 batches: 84.1 / 90.7
 - 4-layer (1024 hidden units), l2=0.001, keep_prob=1.0, 24000 batches, weight decay: 90.5 / 95.7
 - 4-layer (2048, 1024, 512 hidden units), l2=0.001, keep_prob=1.0, 24000 batches, weight decay: 90.4 / 95.5